In [1]:
# 建立实验使用的数据表

import numpy as np
import pandas as pd


def loadData():
    # 字典users的键表示不同用户的名字，值为一个评分字典，评分字典的键值对表示某物品被当前用户的评分
    # 现实场景中，用户对物品的评分会比较稀疏
    users = {'Alice': {'A': 5, 'B': 3, 'C': 4, 'D': 4},
             'user1': {'A': 3, 'B': 1, 'C': 2, 'D': 3, 'E': 3},
             'user2': {'A': 4, 'B': 3, 'C': 4, 'D': 3, 'E': 5},
             'user3': {'A': 3, 'B': 3, 'C': 1, 'D': 5, 'E': 4},
             'user4': {'A': 1, 'B': 5, 'C': 5, 'D': 2, 'E': 1}
             }
    # Alice对A项目的评分是5，对B项目的评分是3
    return users

In [2]:
# 计算用户相似性矩阵
user_data=loadData()
similarity_matrix=pd.DataFrame(
    np.identity(len(user_data)),
    index=user_data.keys(),
    columns=user_data.keys(),
)

In [6]:
# 遍历每条用户-物品评分数据
for u1, items1 in user_data.items():
    # u1：用户的名字（键）
    # items1：该用户对各个项目的评分（值）
    for u2, items2 in user_data.items():
        if u1==u2: # 如果u1和u2是同一个人，不计算用户与自己的相似度
            continue
        vec1,vec2=[],[] # 用户存储两个用户对相同项目的评分
        for item, rating1 in items1.items(): 
            # item：项目
            # rating1：u1用户对该项目的评分
            rating2=items2.get(item,-1) # 获取评分，如果没有的话就是-1
            if rating2==-1: # 如果u2沒有对其评分，就跳过当前项目的比较
                continue
            vec1.append(rating1) # 将u1用户的评分添加到vec1列表中
            vec2.append(rating2) # 将u2用户的评分添加到vec2列表中
        # 计算不同用户之间的皮尔逊相关系数
        similarity_matrix[u1][u2]=np.corrcoef(vec1,vec2)[0][1]
        # np.corrcoef(vec1,vec2)返回一个相关系数矩阵
        # [0][1]和[1][0]位置的键是两个向量的相关系数
        # 这个值被存储在similarity_matrix字典中

print(similarity_matrix)

          Alice     user1     user2     user3     user4
Alice  1.000000  0.852803  0.707107  0.000000 -0.792118
user1  0.852803  1.000000  0.467707  0.489956 -0.900149
user2  0.707107  0.467707  1.000000 -0.161165 -0.466569
user3  0.000000  0.489956 -0.161165  1.000000 -0.641503
user4 -0.792118 -0.900149 -0.466569 -0.641503  1.000000


In [10]:
# 计算与Alice最为相似的num个用户
target_user='Alice'
num=2
sim_users=similarity_matrix[target_user].sort_values(ascending=False)[1:num+1].index.tolist()
print(f'与用户{target_user}最相似的{num}个用户为：{sim_users}')

与用户Alice最相似的2个用户为：['user1', 'user2']


In [14]:
# 预测用户Alice对物品E的评分
weighted_scores=0. # 存储加权评分的和
corr_values_sum=0. # 存储所有相关性系数的总和
 # 预测评分的目标物品
target_item='E'
# 基于皮尔逊相关系数预测用户评分
for user in sim_users: # 遍历与目标用户有相似度评分的其他用户
    corr_value=similarity_matrix[target_user][user] # 获取目标用户和当前遍历用户之间的皮尔逊相关系数
    user_mean_rating=np.mean(list(user_data[user].values())) # 计算当前遍历用户对所有物品的平均评分
    
    # 计算加权评分
    # 这里，每个用户的评分减去其平均评分，然后乘以目标用户和当前用户的相关性系数，最后加到 weighted_scores 上。
    weighted_scores+=corr_value*(user_data[user][target_item]-user_mean_rating)
    corr_values_sum+=corr_value

# 计算目标用户对目标物品的预测评分
target_user_mean_rating=np.mean(list(user_data[target_user].values()))
target_item_pred=target_user_mean_rating+weighted_scores/corr_values_sum 
print(f'用户{target_user}对物品{target_item}的预测评分为：{target_item_pred}')

用户Alice对物品E的预测评分为：4.871979899370592
